In [ ]:
# !pip install --upgrade openai
# !pip install --upgrade langchain
# !pip install --upgrade python-dotenv

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [3]:
possible_intents = [
    "CreateObject",
    "List",
    "Report",
    "ReviewObject",
    "Workflow"
]

In [4]:
possible_objects = [
    "Bank account",
    "Bank reconciliation",
    "Contact",
    "Customer",
    "Entry",
    "Journal",
    "Maintenance",
    "Owner",
    "Payable",
    "Property",
    "Prospect",
    "Accounts Receivable",
    "Tenant",
    "Units",
    "Vendor",
]

In [5]:
len(possible_intents), len(possible_objects)

(5, 15)

In [6]:
intent_schema = ResponseSchema(
    name="FirstOrderIntent",
    description="The firstOrder intent of the real-state statement"
)
object_schema = ResponseSchema(
    name="SecondOrderIntent",
    description="The secondOrder intent of the real-state statement"
)

entities_schema = ResponseSchema(
    name="Entities",
    description="List of all entities in the real-estate statement. Separate each entity by comma."
)
response_schemas = [intent_schema, object_schema, entities_schema]

In [7]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [8]:
template_string = """You are a helpful real-state assistant that can detect the firstOrder intent, secondOrder intent \
and entities from the statement that is delimited by triple backticks. 

Classify the most appropriate firstOrder intent from the following list of 5 firstOrder intents: [{possible_intents}].

And classify the most appropriate secondOrder intent from the following list of 15 secondOrder objects: [{possible_objects}].

Finally, List all the entities from the same statement. Entities takes on multiple values where values are exact word/words\
form the statement.

{format_instructions}

Here are few sample input and output for your reference:
1. "Run a AR report for Cityview towers as of today",
	"FirstOrderIntent": "Report",
	"SecondOrderIntent": "Accounts Receivable",
    "Entities": "Cityview towers, today"

2. "List all customers in New York City"
	"FirstOrderIntent": "List",
	"SecondOrderIntent": "Customer",
    "Entities": "New York City"

3. "What is Michael Smiths current rent"
	"FirstOrderIntent": "ReviewObject",
	"SecondOrderIntent": "Tenant",
    "Entities": "Michael Smith, rent"

4. "What are the available units in Avella Village?"
	"FirstOrderIntent": "List",
	"SecondOrderIntent": "Units",
    "Entities": "Avella Village"

5. "Leaking toilet unit 101 in Boardwalk by Windsor"
	"FirstOrderIntent": "Workflow",
	"SecondOrderIntent": "Maintenance",
    "Entities": "Leaking toilet, Unit 101, Boardwalk by Windsor"
    
6. "Create a new contact entry for a potential buyer interested in Property XYZ."
	"FirstOrderIntent": "CreateObject",
	"SecondOrderIntent": "Contact",
    "Entities": "entry, buyer, Property XYZ"

7. "Create a new customer entry for Property ABC."
	"FirstOrderIntent": "CreateObject",
	"SecondOrderIntent": "Customer",
    "Entities": "entry, Property ABC"

8. "Create a new journal entry for property management fees incurred in Building A."
	"FirstOrderIntent": "CreateObject",
	"SecondOrderIntent": "Entry",
    "Entities": "management fees, building A"

9. "Run a bank account report for Property XYZ as of today."
	"FirstOrderIntent": "Report",
	"SecondOrderIntent": "Bank account",
    "Entities": "Property XYZ, today"

10. "Run a bank reconciliation report for City Heights Apartments as of today."
	"FirstOrderIntent": "Report",
	"SecondOrderIntent": "Bank reconciliation",
    "Entities": "City Heights Apartments, today"

11. "Review the maintenance log for Property ABC to assess the repair history."
	"FirstOrderIntent": "ReviewObject",
	"SecondOrderIntent": "Maintenance",
    "Entities": "property ABC, assess, repair history"
    
12. "Review the contact details and preferences of a potential prospect for Property ABC."
	"FirstOrderIntent": "ReviewObject",
	"SecondOrderIntent": "Prospect",
    "Entities": "contact, preferences, property ABC" 

13. "Review the contact details for the vendor of Cityview towers."
	"FirstOrderIntent": "ReviewObject",
	"SecondOrderIntent": "Vendor",
    "Entities": "contact, Cityview towers"

14. "Initiate a bank account workflow for Property XYZ."
	"FirstOrderIntent": "Workflow",
	"SecondOrderIntent": "Bank account",
    "Entities": "Property XYZ"

15. "Initiate a contact workflow for Property XYZ."
	"FirstOrderIntent": "Workflow",
	"SecondOrderIntent": "Contact",
    "Entities": "Property XYZ"

16. "Initiate a unit workflow for Property XYZ."
	"FirstOrderIntent": "Workflow",
	"SecondOrderIntent": "Unit",
    "Entities": "Property XYZ"
    
17. "List all owner properties in New York City."
	"FirstOrderIntent": "List",
	"SecondOrderIntent": "Owner",
    "Entities": "properties, New York city"

18. "List all journal entries related to property expenses for tax reporting purposes."
	"FirstOrderIntent": "List",
	"SecondOrderIntent": "Journal",
    "Entities": "entries, property expenses, tax reporting purposes"

19. "Create a new payable entry for Property XYZ."
	"FirstOrderIntent": "CreateObject",
	"SecondOrderIntent": "Payable",
    "Entities": "entry, Property XYZ"

20. "Create a new listing for a beachfront property in Malibu."
	"FirstOrderIntent": "CreateObject",
	"SecondOrderIntent": "Property",
    "Entities": "beachfront, Malibu"

"""

In [9]:
system_message_prompt = SystemMessagePromptTemplate.from_template(template_string)
human_template = "statement: ```{statement}```"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [10]:
chat_prompt

ChatPromptTemplate(input_variables=['possible_intents', 'format_instructions', 'statement', 'possible_objects'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'possible_intents', 'possible_objects'], output_parser=None, partial_variables={}, template='You are a helpful real-state assistant that can detect the firstOrder intent, secondOrder intent and entities from the statement that is delimited by triple backticks. \n\nClassify the most appropriate firstOrder intent from the following list of 5 firstOrder intents: [{possible_intents}].\n\nAnd classify the most appropriate secondOrder intent from the following list of 15 secondOrder objects: [{possible_objects}].\n\nFinally, List all the entities from the same statement. Entities takes on multiple values where values are exact word/wordsform the statement.\n\n{format_instructions}\n\nHere are few sample input and output for your reference:\n

In [11]:
# statement = "List all customers in New York City"
# messages = chat_prompt.format_prompt(
#     possible_intents=possible_intents,
#     possible_objects=possible_objects,
#     format_instructions=format_instructions,
#     statement=statement
# ).to_messages()
# chat = ChatOpenAI(temperature=0.0)
# response = chat(messages)
# output_dict = output_parser.parse(response.content)
# output_dict

In [12]:
def classify(statement):
    messages = chat_prompt.format_prompt(
        possible_intents=possible_intents,
        possible_objects=possible_objects,
        format_instructions=format_instructions,
        statement=statement
    ).to_messages()
    chat = ChatOpenAI(temperature=0.0)
    response = chat(messages)
    return output_parser.parse(response.content)

In [13]:
classify("What is Michael Smiths current rent")

{'FirstOrderIntent': 'ReviewObject',
 'SecondOrderIntent': 'Tenant',
 'Entities': 'Michael Smith, rent'}

In [14]:
classify("What are the available units in Avella Village?")

{'FirstOrderIntent': 'List',
 'SecondOrderIntent': 'Units',
 'Entities': 'Avella Village'}

In [15]:
classify("Leaking toilet unit 101 in Boardwalk by Windsor")

{'FirstOrderIntent': 'Workflow',
 'SecondOrderIntent': 'Maintenance',
 'Entities': 'Leaking toilet, unit 101, Boardwalk by Windsor'}

In [16]:
classify("Run a AR report for Cityview towers as of today")

{'FirstOrderIntent': 'Report',
 'SecondOrderIntent': 'Accounts Receivable',
 'Entities': 'Cityview towers, today'}

In [17]:
classify("Create a new journal entry for Cityview towers.")

{'FirstOrderIntent': 'CreateObject',
 'SecondOrderIntent': 'Journal',
 'Entities': 'journal entry, Cityview towers'}

In [18]:
classify("Create a new prospect profile for a potential buyer interested in residential properties.")

{'FirstOrderIntent': 'CreateObject',
 'SecondOrderIntent': 'Prospect',
 'Entities': 'prospect profile, potential buyer, residential properties'}

In [19]:
classify("List all bank reconciliations for Property XYZ.")

{'FirstOrderIntent': 'List',
 'SecondOrderIntent': 'Bank reconciliation',
 'Entities': 'Property XYZ'}

In [20]:
classify("List all tenants residing in Cityview Towers.")

{'FirstOrderIntent': 'List',
 'SecondOrderIntent': 'Tenant',
 'Entities': 'Cityview Towers'}

In [21]:
classify("Review the vendor agreement with the landscaping service provider for Cityview Towers.")

{'FirstOrderIntent': 'ReviewObject',
 'SecondOrderIntent': 'Vendor',
 'Entities': 'vendor agreement, landscaping service provider, Cityview Towers'}